# NASLib Overview
NASLib is framework that was built in order to facilitate neural architecture search (NAS) research and development. Please refer to the slides and to the NAS survey paper for more details. In a high-level NASLib consists of 4 main building blocks which (can) interact with each other:

- search spaces (cell search space, hierarchical, ...)
- optimizers (one-shot/weight-sharing optimizers, black-box optimizers)
- predictors (performance estimators that given an architecture as input, output its performance)
- evaluators (run the architecture search loop and the final network training pipeline)

Please follow the installation and setup guide in the main README of the repository so that you are able to ```import naslib``` without any errors.

# Search Spaces in NASLib
The search space representation is of primary importance for NASLib in ensuring that optimizers and search spaces can be combined in a variety of ways. The predominant way of representing NAS search spaces is the directed acyclic graph (DAG). In order to accomplish the aforementioned functionality of search spaces and computational graphs, we inherit in our basic graph classes from both *PyTorch* and *NetworkX*. The latter is a well-maintained and tested Python package for graph creation and manipulation, where node and edge attributes can be arbitrary Python objects. This framework allows us to represent multiple layers of graphs on top of the computational graph, allowing us to treat nodes and edges both as primitive operations (e.g. convolution), but also nested graph-structures such as a DARTS cell, to create e.g. macro architectures of stacked cells. NetworkX allows to easily construct the search space via ```add_node, remove_node, add_edge, remove_edge```, or traverse the topologically sorted graph in the forward pass of the PyTorch module using ```networkx.algorithms.dag.topological_sort```.

## Case study: NASBench-201
Case study: NAS-Bench-201
The [NAS-Bench-201](https://arxiv.org/abs/2001.00326) is a tabular benchmark, i.e. a benchmark where you can simply query (already has been trained) the performance and other metrics of a specific architecture in the search space given that as an input. Its search space consists of a single normal cell which is replicated multiple times in a macro architecture interleaved by manually defined resnet-like reduction cells. The cell topology is fixed in the cell and consists of:

1 input, 2 intermediate and 1 output node;
a summation operation on each of the intermediate and output nodes;
5 operation choices in each of the edges connecting 2 nodes
- 'none'
- 'skip_connect'
- 'nor_conv_1x1'
- 'nor_conv_3x3'
- 'avg_pool_3x3'

In [1]:
    from naslib.search_spaces import NasBench201SearchSpace as NB201

    # instantiate the search space object
    search_space = NB201()

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0


Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`


device: cuda:0
device: cuda:0


Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`


## Black-box optimizers in NASLib
After learning about the search space object, now we can add the other component of NAS: the NAS optimizer which you will use to search for an optimal architecture in that search space. A search space graph object can be interpreted in different ways depending on the type of optimizer being used. Here is the point where the search space and optimizer objects interact by parsing information from each other.

In [2]:
# import some utilities and parse the configuration file
import logging

from naslib.utils import utils, setup_logger, get_dataset_api

# This will read the parameters from the default yaml configuration file, which in this 
# case is located in NASLib/naslib/benchmarks/nas_predictors/discrete_config.yaml.
# You do not have to change this but you can play around with its parameters.
config = utils.get_config_from_args(config_type="nas_predictor")
utils.set_seed(config.seed)
utils.log_args(config)

logger = setup_logger(config.save + "/log.log")
logger.setLevel(logging.INFO)

In [3]:
from naslib.optimizers import RegularizedEvolution as RE

# instantiate the optimizer object using the configuration file parameters
optimizer = RE(config)

After parsing the configuration file and instantiating the NAS optimizer and search space objects, we have to adapt the search space based on the optimizer type. A black-box optimizer such as Random Search will sample single architectures using the sample_random_architecture method of the search space object (e.g. by sampling one operation at each graph edge from the operation choices in NAS-Bench-201) throughout the optimization process. On the other hand most one-shot optimizers, such as DARTS, will interpret a set of operation choices on an edge as a MixedOp and assign an appropriate number of architectural weights (between 0 and 1, such that the sum is 1) to the outputs of each operation in order to obtain the continuous relaxation.

Download the NAS-Bench-201 data from https://drive.google.com/file/d/17EBlTidimMaGrb3fE0APbljJl-ocgfs4/view?usp=sharing and place it in ```NASLib/naslib/data/```

In [4]:
# this will load the NAS-Bench-201 data (architectures and their accuracy, runtime, etc).
dataset_api = get_dataset_api(config.search_space, config.dataset)

# adapt the search space to the optimizer type
optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

Now the only step left is to run the search. For this we will use the ```Trainer``` object in NASLib

In [5]:
from naslib.defaults.trainer import Trainer

# since the optimizer has parsed the information of the search space, we do not need to pass the search
# space object to the trainer when instantiating it.
trainer = Trainer(optimizer, config, lightweight_output=True)

[11/15 22:07:24 nl.defaults.trainer]: param size = 0.000000MB


In [6]:
# call only a method to run the search for the number of iterations specified in the yaml configuration file.
trainer.search()

[11/15 22:07:32 nl.defaults.trainer]: Start training
[11/15 22:07:32 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
WARNING [11/15 22:07:32 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
WARNING [11/15 22:07:32 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
[11/15 22:07:32 nl.optimizers.discrete.re.optimizer]: Population size 1
[11/15 22:07:32 nl.defaults.trainer]: Epoch 0, Anytime results: {'cifar10-valid': {'train_losses': [1.8448507370758056, 1.4482701889801026, 1.2033821305465697, 1.0578395529174804, 0.9672217133712768, 0.9047809088516235, 0.8624664130783081, 0.8102531592178345, 0.7739742643928528, 0.7372287948799133, 0.7112728433036805, 0.6888134226989746, 0.656216381034851, 0.6430078009033203, 0.6244383986091614, 0.6145832547187805, 0.6030634831237793, 

[11/15 22:07:32 nl.defaults.trainer]: Epoch 0 done. Train accuracy (top1, top5): 99.22400, 0.00000, Validation accuracy: 86.77000, 0.00000
[11/15 22:07:32 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:33 nl.defaults.trainer]: Epoch 1 done. Train accuracy (top1, top5): 99.22400, 0.00000, Validation accuracy: 86.77000, 0.00000
[11/15 22:07:33 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:33 nl.defaults.trainer]: Epoch 2 done. Train accuracy (top1, top5): 99.22400, 0.00000, Validation accuracy: 86.77000, 0.00000
[11/15 22:07:33 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:33 nl.defaults.trainer]: Epoch 3 done. Train accuracy (top1, top5): 99.96000, 0.00000, Validation accuracy: 89.47000, 0.00000
[11/15 22:07:33 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:33 nl.d

[11/15 22:07:37 nl.defaults.trainer]: Epoch 26 done. Train accuracy (top1, top5): 99.97200, 0.00000, Validation accuracy: 89.89000, 0.00000
[11/15 22:07:37 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:38 nl.defaults.trainer]: Epoch 27 done. Train accuracy (top1, top5): 99.97200, 0.00000, Validation accuracy: 89.89000, 0.00000
[11/15 22:07:38 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:38 nl.defaults.trainer]: Epoch 28 done. Train accuracy (top1, top5): 99.97200, 0.00000, Validation accuracy: 89.89000, 0.00000
[11/15 22:07:38 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[11/15 22:07:38 nl.defaults.trainer]: Epoch 29 done. Train accuracy (top1, top5): 99.97200, 0.00000, Validation accuracy: 89.89000, 0.00000
[11/15 22:07:38 nl.defaults.trainer]: Epoch 30 done. Train accuracy (top1, top5): 99.97200, 0.00000, Validation accuracy: 

[11/15 22:07:43 nl.defaults.trainer]: Epoch 57 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:43 nl.defaults.trainer]: Epoch 58 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:43 nl.defaults.trainer]: Epoch 59 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:44 nl.defaults.trainer]: Epoch 60 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:44 nl.defaults.trainer]: Epoch 61 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:44 nl.defaults.trainer]: Epoch 62 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:44 nl.defaults.trainer]: Epoch 63 done. Train accuracy (top1, top5): 99.98800, 0.00000, Validation accuracy: 91.29000, 0.00000
[11/15 22:07:44 nl.d

[11/15 22:07:48 nl.defaults.trainer]: Epoch 90 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:48 nl.defaults.trainer]: Epoch 91 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.defaults.trainer]: Epoch 92 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.defaults.trainer]: Epoch 93 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.defaults.trainer]: Epoch 94 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.defaults.trainer]: Epoch 95 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.defaults.trainer]: Epoch 96 done. Train accuracy (top1, top5): 99.98400, 0.00000, Validation accuracy: 91.31000, 0.00000
[11/15 22:07:50 nl.d

In [7]:
# After the search is done, we want to evaluate the test performance of
# the best architecture found using the validation set.
trainer.evaluate(dataset_api=dataset_api)

[11/15 22:07:58 nl.defaults.trainer]: Start evaluation
[11/15 22:07:58 nl.defaults.trainer]: loading model from file run/cifar10/nas_predictors/nasbench201/var_sparse_gp/1000/search/model_final.pth
[11/15 22:07:58 nl.defaults.trainer]: Final architecture:
Graph makrograph-0.4395186, scope None, 20 nodes
[11/15 22:07:58 nl.defaults.trainer]: Queried results (Metric.TEST_ACCURACY): 91.31
